### Import modules from fjagepy and unetpy 

In [ ]:
from fjagepy import *
from unetpy import *

### Some useful functions for modem operations 

In [ ]:
# Open a connection to modem

def modemOpen(ip, port):
    try:
        modem = org_arl_fjage_remote.Gateway(ip, port)
        print("Connected!")
        return modem
    except Exception as e:
        print("Exception:" + str(e))
        sys.exit(0)

# Close the connection to modem        
def modemClose(modem):
    modem.shutdown()

# Get the range to the modem with specified address
def modemgetRange(modem, address):
    rangeReq = org_arl_unet_phy.RangeReq()
    rangeReq.to = address
    rangeReq.recipient = 'ranging'
    rsp = modem.request(rangeReq, 10000)
    if rsp and rsp.perf == org_arl_fjage.Performative.AGREE:
        rangeRec = modem.receive(org_arl_unet_phy.RangeNtf, 10000)
        if rangeRec == None:
            print("Error getting range!")
        else:
            return rangeRec.range

# Record a signal        
def modemRecord(modem, recLen):
    recReq = org_arl_unet_bb.RecordBasebandSignalReq()
    recReq.recLen = recLen
    recReq.recipient = 'phy'
    rsp = modem.request(recReq, 1000)
    if rsp and rsp.perf == org_arl_fjage.Performative.AGREE:
        print("DEBUG")
        rec = modem.receive(org_arl_unet_bb.RxBasebandSignalNtf, 10000)
        return rec.signal

# Transmit a frame    
def modemFrameTx(modem, data):
    txReq = org_arl_unet_phy.TxFrameReq()
    txReq.recipient = 'phy'
    txReq.data = data
    rsp = modem.request(txReq, 5000)

# Transmit a signal    
def modemSignalTx(modem, data):
    txSignal = org_arl_unet_bb.TxBasebandSignalReq()
    txSignal.recipient = 'phy'
    txSignal.preamble = 3
    txSignal.dac = 0
    txSignal.channels = 1
    txSignal.fc = -1
    txSignal.fs = -1
    txSignal.signal = data
    rsp = modem.request(txSignal, 5000)

###  Example python scripts using the above functions 

###### <span style="color:red"> Transmit a frame using TxFrameReq </span> 

In [ ]:
# modem = modemOpen('192.168.1.179', 1100)
modem = modemOpen('localhost', 1100)
modemFrameTx(modem, [1,1])

###### <span style="color:red"> Transmit a signal </span> 

In [ ]:
# modem = modemOpen('192.168.1.109', 1100)
modemSignalTx(modem, [1,1,1,1,1,1])

###### <span style="color:red"> Ranging </span> 

In [ ]:
# modem = modemOpen('192.168.1.175', 1100)
modem.subscribe(modem.topic("ranging__ntf"))
modemgetRange(modem, 9)

###### <span style="color:red"> Record the signal and plot </span> 

In [ ]:
import matplotlib.pyplot as plt

# modem = modemOpen('192.168.1.175', 1100)
modem.subscribe(modem.topic("phy__ntf"))
signal = modemRecord(modem, 8190)
plt.plot(signal)
plt.show()

###### <span style="color:red"> Close the connection to modem </span> 

In [ ]:
modemClose(modem)